In [1]:
import pandas as pd
import numpy as np
import re

In [2]:
data = pd.read_csv('thanksgiving.csv', encoding="Latin-1")

In [3]:
data.head(3)

,RespondentID,Do you celebrate Thanksgiving?,What is typically the main dish at your Thanksgiving dinner?,What is typically the main dish at your Thanksgiving dinner? - Other (please specify),How is the main dish typically cooked?,How is the main dish typically cooked? - Other (please specify),What kind of stuffing/dressing do you typically have?,What kind of stuffing/dressing do you typically have? - Other (please specify),What type of cranberry saucedo you typically have?,What type of cranberry saucedo you typically have? - Other (please specify),...,Have you ever tried to meet up with hometown friends on Thanksgiving night?,"Have you ever attended a ""Friendsgiving?""",Will you shop any Black Friday sales on Thanksgiving Day?,Do you work in retail?,Will you employer make you work on Black Friday?,How would you describe where you live?,Age,What is your gender?,How much total combined money did all members of your HOUSEHOLD earn last year?,US Region
0,4337954960,Yes,Turkey,NaN,Baked,NaN,Bread-based,NaN,None,NaN,...,Yes,No,No,No,NaN,Suburban,18 - 29,Male,"$75,000 to $99,999",Middle Atlantic
1,4337951949,Yes,Turkey,NaN,Baked,NaN,Bread-based,NaN,Other (please specify),Homemade cranberry gelatin ring,...,No,No,Yes,No,NaN,Rural,18 - 29,Female,"$50,000 to $74,999",East South Central
2,4337935621,Yes,Turkey,NaN,Roasted,NaN,Rice-based,NaN,Homemade,NaN,...,Yes,Yes,Yes,No,NaN,Suburban,18 - 29,Male,"$0 to $9,999",Mountain


In [4]:
data.columns

Index(['RespondentID', 'Do you celebrate Thanksgiving?',
       'What is typically the main dish at your Thanksgiving dinner?',
       'What is typically the main dish at your Thanksgiving dinner? - Other (please specify)',
       'How is the main dish typically cooked?',
       'How is the main dish typically cooked? - Other (please specify)',
       'What kind of stuffing/dressing do you typically have?',
       'What kind of stuffing/dressing do you typically have? - Other (please specify)',
       'What type of cranberry saucedo you typically have?',
       'What type of cranberry saucedo you typically have? - Other (please specify)',
       'Do you typically have gravy?',
       'Which of these side dishes aretypically served at your Thanksgiving dinner? Please select all that apply. - Brussel sprouts',
       'Which of these side dishes aretypically served at your Thanksgiving dinner? Please select all that apply. - Carrots',
       'Which of these side dishes aretypically served

###### Separating the people that celebrate thanksgiving for further analysis

In [5]:
count_celebrate = data['Do you celebrate Thanksgiving?'].value_counts()

In [6]:
count_celebrate

Yes    980
No      78
Name: Do you celebrate Thanksgiving?, dtype: int64

In [7]:
data_celebrate = data[data['Do you celebrate Thanksgiving?'] == 'Yes']

###### counting the number of the unique main dishes people eat in tnxgiving dinner

In [8]:
data_celebrate['What is typically the main dish at your Thanksgiving dinner?'].value_counts()

Turkey                    859
Other (please specify)     35
Ham/Pork                   29
Tofurkey                   20
Chicken                    12
Roast beef                 11
I don't know                5
Turducken                   3
Name: What is typically the main dish at your Thanksgiving dinner?, dtype: int64

###### Checking if the people who eat Tofurkey, have gravy as well or not?

In [9]:
#data_celebrate[['What is typically the main dish at your Thanksgiving dinner?','Do you typically have gravy?']][data_celebrate['What is typically the main dish at your Thanksgiving dinner?']=='Tofurkey']

###### checking if a person ate one of  the  Apple, Pumpkin, Pecan for desert 

In [10]:
ate_pies = data_celebrate[pd.isnull(data_celebrate['Which type of pie is typically served at your Thanksgiving dinner? Please select all that apply. - Apple']) &\
                         pd.isnull(data_celebrate['Which type of pie is typically served at your Thanksgiving dinner? Please select all that apply. - Pumpkin']) &\
                                        pd.isnull(data_celebrate['Which type of pie is typically served at your Thanksgiving dinner? Please select all that apply. - Pecan'])]

In [11]:
#ate_pies

##### analyze the age column
###### replace the average age of each category instead of the range

In [12]:
age_categs = data_celebrate['Age'].value_counts()
age_categs

45 - 59    269
60+        258
30 - 44    235
18 - 29    185
Name: Age, dtype: int64

In [13]:
def transform_age(age):
    if pd.isnull(age):
        return None
    elif age == '60+':
        age_parsed = age[0:2]
        return int(age_parsed)
    else:
        age_parsed = age.split(' ')
        #print((int(age_parsed[0])+int(age_parsed[2]))/2)
        mean_age = (int(age_parsed[0])+int(age_parsed[2]))/2
        return mean_age

In [14]:
data_age_tranformed = data_celebrate['Age'].apply(transform_age)

In [15]:
data_age_tranformed.describe()

count    947.000000
mean      44.889652
std       13.475203
min       23.500000
25%       37.000000
50%       52.000000
75%       60.000000
max       60.000000
Name: Age, dtype: float64

### conclusion: this cannot be a good age analysis, since only the age range was provided

###### convert the houshold earnings to numerical values and get the statistical data

In [16]:
data_celebrate['How much total combined money did all members of your HOUSEHOLD earn last year?'].value_counts()
data_celebrate['How much total combined money did all members of your HOUSEHOLD earn last year?'].unique()

array(['$75,000 to $99,999', '$50,000 to $74,999', '$0 to $9,999',
       '$200,000 and up', '$100,000 to $124,999', '$25,000 to $49,999',
       'Prefer not to answer', '$10,000 to $24,999',
       '$175,000 to $199,999', '$150,000 to $174,999',
       '$125,000 to $149,999', nan], dtype=object)

In [17]:
def transform_earnings(earn):
    if pd.isnull(earn):
        return None
    elif earn =='Prefer not to answer':
        return None
    elif earn == '$200,000 and up':
        return 200000
    else:
        earn_split = earn.split(' ')
        earning_list = []
        if earn_split[0] != '$0':
            for item in [earn_split[0], earn_split[2]]:
                earning = item[1:]
                m = re.search(',',earning)
                earning = earning[:m.start()] + earning[m.end():]
                earning_list.append(int(earning))
        else:
            earning_list.append(0)
            earning = earn_split[2][1:]
            m = re.search(',',earning)
            earning = earning[:m.start()] + earning[m.end():]
            earning_list.append(int(earning))
        earning_mean = sum(earning_list)/2
        return earning_mean 

In [18]:
data_earnings = data_celebrate['How much total combined money did all members of your HOUSEHOLD earn last year?'].apply(transform_earnings)

In [19]:
data_earnings.describe()

count       829.000000
mean      86486.276840
std       57789.467567
min        4999.500000
25%       37499.500000
50%       87499.500000
75%      112499.500000
max      200000.000000
Name: How much total combined money did all members of your HOUSEHOLD earn last year?, dtype: float64

##### Find the realtionhip between the travel distance and household income

In [51]:
data_celebrate['How far will you travel for Thanksgiving?'].unique()
data_celebrate['How far will you travel for Thanksgiving?'].value_counts()

Thanksgiving is happening at my home--I won't travel at all                         396
Thanksgiving is local--it will take place in the town I live in                     276
Thanksgiving is out of town but not too far--it's a drive of a few hours or less    197
Thanksgiving is out of town and far away--I have to drive several hours or fly       82
Name: How far will you travel for Thanksgiving?, dtype: int64

###### create a new df with travel distance info and houshold earning

In [53]:
travel_distance = data_celebrate['How far will you travel for Thanksgiving?'].fillna('')
income_distance_df = pd.DataFrame({'Household income':data_earnings, 'Travel Distance':travel_distance})
#income_distance_df

###### comparing the travel distance for the people with more and less than 15,000 houshold income

In [68]:
higher_income_travel = income_distance_df['Travel Distance'][income_distance_df['Household income']>15000].value_counts()
higher_income_travel_percentage = (higher_income_travel/higher_income_travel.sum())*100
higher_income_travel_percentage

Thanksgiving is happening at my home--I won't travel at all                         41.956242
Thanksgiving is local--it will take place in the town I live in                     28.571429
Thanksgiving is out of town but not too far--it's a drive of a few hours or less    20.978121
Thanksgiving is out of town and far away--I have to drive several hours or fly       8.494208
Name: Travel Distance, dtype: float64

In [69]:
lower_income_travel = income_distance_df['Travel Distance'][income_distance_df['Household income']<=15000].value_counts()
lower_income_travel_percentage = (lower_income_travel/lower_income_travel.sum())*100
lower_income_travel_percentage

Thanksgiving is happening at my home--I won't travel at all                         40.384615
Thanksgiving is local--it will take place in the town I live in                     28.846154
Thanksgiving is out of town but not too far--it's a drive of a few hours or less    23.076923
Thanksgiving is out of town and far away--I have to drive several hours or fly       7.692308
Name: Travel Distance, dtype: float64

### Conclusion: the percentage of travel distance is almost the same between the lower and higher income people. However, the total number of people with higher than 15,000 salary is much more than people with less than 15,000 earning